# Liver Model Construction: Notebook

In [2]:
import os
import re
import warnings
from cobra.io import *
from cobra.io.json import *
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv
import altair as alt
from minspan.minspan import minspan,nnz
# from minspan import minspan,nnz

MASSpy version: 0.1.6


In [2]:
import sys
# from minspan import minspan,nnz
print(sys.version)

3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]


In [3]:
maps_dir = os.path.abspath("maps")
data_dir = os.path.abspath("data")
dir = os.path.abspath("")
model_dir = os.path.abspath("models")
minspan_dir= os.path.abspath("minspans_csv")
# extra_dir=os.path.abspath("..")
# minspan_dir

### Getting reactions and metabolite data from RECON 3D

In [4]:
# making dataframe of metabolites

csv_met = os.path.join(data_dir,"glycolysis_gluconeogenesis_met_df.csv")
met_csv_df = pd.read_csv(csv_met,index_col="id")
metabolite_info=met_csv_df.to_dict(orient='index')



# making dataframe of reactions
csv_rxn = os.path.join(data_dir,"glycolysis_gluconeogenesis_reaction_df.csv")
rxn_csv_df = pd.read_csv(csv_rxn,index_col="id") 
reaction_info = rxn_csv_df.to_dict(orient='index')

### Loading cobra model

In [5]:
# Initiate empty model
trial= cobra.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-01


In [6]:
#  Add all the remaining metabolites involved in the pathway
for met_id, info in metabolite_info.items():
    met = cobra.Metabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    trial.add_metabolites(met)

In [7]:
# Loading reaction data as JSON file to maintain data types as dictionaries 
rxn_json = os.path.join(data_dir,"glycolysis_gluconeogenesis_reaction_df.json")
with open(rxn_json) as json_file:
    rxn_data = json.load(json_file)

In [ ]:
# rxn_data

In [8]:
for reaction, info in rxn_data.items():
    reaction_obj = cobra.Reaction(reaction)
    reaction_obj.id=reaction
    reaction= reaction_obj.id
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
    trial.add_reaction(reaction_obj)
    temp=info['metabolites']
    reaction_obj.add_metabolites(temp)
    print(reaction)

HEX1
PGI
FBP
PFK
FBA
TPI
GAPD
PGK
PGM
ENO
PEPCKm
PYK
PCm
LDH_L
PYRt2m
ADK1
ATPM


In [9]:
trial.reactions.HEX1

Reaction identifier,HEX1
Name,Hexokinase (D-glucose:ATP)
Memory address,0x024948f5d970
Stoichiometry,atp_c + glc__D_c --> adp_c + g6p_c + h_c ATP C10H12N5O13P3 + D-Glucose --> ADP C10H12N5O10P2 + D-Glucose 6-phosphate + H+
GPR,
Lower bound,0.0
Upper bound,1000.0


In [10]:
#adding DM_NADH 
nadhm = {'nadh_c': -1, 'h_c': 1, 'nad_c': 1}
rxn = cobra.Reaction('DM_nadh')
trial.add_reaction(rxn)
rxn.add_metabolites(nadhm)




peptm = {'pep_m': -1, 'pi_c': -1, 'pi_m': 1, 'pep_c':1}
rxn2 = cobra.Reaction('PEPtm')
trial.add_reaction(rxn2)
rxn2.add_metabolites(peptm)



g6pase = {'g6p_c': -1, 'h2o_c': -1, 'pi_c': 1, 'glc__D_c':1}
rxn3 = cobra.Reaction('G6PP')
trial.add_reaction(rxn3)
rxn3.add_metabolites(g6pase)


## add PEPtm and g6paseb


In [11]:
trial.reactions.DM_nadh

Reaction identifier,DM_nadh
Name,
Memory address,0x024948f8fac0
Stoichiometry,nadh_c --> h_c + nad_c Nicotinamide adenine dinucleotide - reduced --> H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [12]:
#Mass balance check
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

HEX1 {'charge': -1.0}
PGI {}
FBP {'charge': 2.0}
PFK {'charge': -1.0}
FBA {}
TPI {}
GAPD {'charge': -3.0}
PGK {}
PGM {}
ENO {}
PEPCKm {}
PYK {'charge': 1.0}
PCm {'charge': 1.0}
LDH_L {'charge': -1.0}
PYRt2m {}
ADK1 {}
ATPM {'charge': 1.0}
DM_nadh {'charge': 1}
PEPtm {}
G6PP {'charge': 2}


In [13]:
trial.metabolites.h_c.charge= 1
trial.metabolites.h_m.charge= 1
# trial.metabolites.h_e.charge= 1
trial.metabolites.pi_c.charge= -2
trial.metabolites.pi_m.charge= -2

In [14]:
#Mass balance check
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

HEX1 {}
PGI {}
FBP {}
PFK {}
FBA {}
TPI {}
GAPD {}
PGK {}
PGM {}
ENO {}
PEPCKm {}
PYK {}
PCm {}
LDH_L {}
PYRt2m {}
ADK1 {}
ATPM {}
DM_nadh {'charge': 2}
PEPtm {}
G6PP {}


In [15]:
for met in ['glc__D_c', 'pyr_c', 'h2o_c', 'h_c', 'pi_c','lac__L_c']:
    rxn = cobra.Reaction('EX_%s' % met)
    trial.add_reaction(rxn)
    rxn.add_metabolites({met: -1})
    if met == 'glc__D_c':
        rxn.lower_bound = -1 # by convention negative exchange flux = uptake
    elif met == 'pyr_c':
        rxn.lower_bound = 0
    else:
        rxn.lower_bound = -1000

In [16]:
# solved_fluxes = minspan(trial, cores=3, verbose=False, timelimit=60)
# from model_dir import minspan
# df = pd.DataFrame(solved_fluxes.copy(), index=[i.id for i in trial.reactions])
# df = df/df.abs().max()
# csv = os.path.join(data_dir,"minspan_trial 1")
# df.to_csv(csv  % model_name)

In [44]:
trial.objective = 'ATPM'
flux_solution = trial.optimize()
flux_solution.fluxes

HEX1           1.0
PGI            1.0
FBP            0.0
PFK            1.0
FBA            1.0
TPI            1.0
GAPD           2.0
PGK           -2.0
PGM           -2.0
ENO            2.0
PEPCKm         0.0
PYK            2.0
PCm            0.0
LDH_L          0.0
PYRt2m         0.0
ADK1           0.0
ATPM           2.0
DM_nadh        2.0
PEPtm          0.0
G6PP           0.0
EX_glc__D_c   -1.0
EX_pyr_c       2.0
EX_h2o_c       0.0
EX_h_c         6.0
EX_pi_c        0.0
EX_lac__L_c    0.0
Name: fluxes, dtype: float64

In [1]:
#View all the reactions that we need build
escher_builder = escher.Builder(
    model=trial,
    map_json=os.path.join(
        maps_dir,"Glycolysis_Gluconeogenesis.json"),highlight_missing=True)

escher_builder

NameError: name 'escher' is not defined

In [19]:
from cobra.io import json
import cobra.test
import os
from os.path import join
from glob import glob



In [20]:

# gly_gluco_json = os.path.join(model_dir,"gly_gluco_json.json")
# cobra.io.save_json_model(trial, gly_gluco_json )

In [21]:
gly_gluco_json = os.path.join(model_dir,"glycolysis_gluconeogenesis.json")
json.save_json_model(model=trial, filename=gly_gluco_json)




In [ ]:
# import cplex as cplex
# import gurobipy as gurobi

In [ ]:
# trial.solver.add("gurobi")
# trial.solver='cplex'
# cobra.util.solver.qp_solvers = ['cplex', 'gurobi']
# trial.solver

In [46]:
for model_file in glob(gly_gluco_json):
    model_name = model_file.split('/')[-1]
    if 'model' not in model_name:
        continue
    print(model_name)
    model= load_json_model(model_file)
    if 'NADPHM' in model.reactions:
        model.remove_reactions(['NADPHM'])
    # media = ['EX_lac__L_c', 'EX_pyr_c', 'EX_octa_c', 'EX_gln__L_c', 'EX_acetone_c', 'EX_bhb_c',
    #          'EX_glu__L_c', 'EX_ser__L_c', 'EX_cys__L_c', 'EX_gly_c', 'EX_ala__L_c', 'EX_so3_c',
    #         'EX_etoh_c', 'EX_fru_c']
    media = ['EX_glc__D_c']
    for met in media:
        if met in model.reactions:
            model.reactions.get_by_id(met).lower_bound = -1000.

    rxns = [i.id for i in model.reactions]
    #removed blocke reactions because it included ['PEPCKm', 'PCm', 'PYRt2m', 'ADK1', 'PEPtm', 'EX_h2o_c', 'EX_pi_c']
    # blocked = cobra.flux_analysis.find_blocked_reactions(model)
    # print(blocked)
    # model.remove_reactions(blocked)

    solved_fluxes = minspan(model, cores=3, verbose=False, timelimit=60)
    
    df = pd.DataFrame(solved_fluxes.copy(), index=[i.id for i in model.reactions])
    df = df/df.abs().max()
    csv_dir = os.path.join(minspan_dir,"gly_gluco_csv.csv")
    df.to_csv(csv_dir)

c:\Users\sicil\LiverModel\models\glycolysis_gluconeogenesis.json
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpues5kae8.lp
Reading time = 0.01 seconds
: 31 rows, 52 columns, 174 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpkt54gh_t.lp
Reading time = 0.01 seconds
: 83 rows, 78 columns, 318 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpu8tuon9g.lp
Reading time = 0.01 seconds
: 83 rows, 78 columns, 318 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmp7fo_h3r2.lp
Reading time = 0.02 seconds
: 83 rows, 78 columns, 318 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpxmvat6si.lp
Reading time = 0.01 seconds
: 83 rows, 78 columns, 318 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpo_vb7gm4.lp
Reading time = 0.01 seconds
: 83 rows, 78 columns, 318 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpukz

In [22]:
from cobra_dict import metabolite_to_dict as metabolite_to_dict
from cobra_dict import reaction_to_dict as reaction_to_dict

In [23]:
met_df=pd.DataFrame()
for met in trial.metabolites:
    cobra_met= trial.metabolites.get_by_id(met.id)
    m=metabolite_to_dict(cobra_met)
    df_2=pd.DataFrame.from_dict(m,orient='index')
    df_2=df_2.T
    met_df=met_df.append(df_2)

met_df=met_df.set_index('id')
met_df

,name,charge,formula,compartment
id,,,,
glc__D_c,D-Glucose,0,C6H12O6,c
g6p_c,D-Glucose 6-phosphate,-2,C6H11O9P,c
adp_c,ADP C10H12N5O10P2,-3,C10H12N5O10P2,c
atp_c,ATP C10H12N5O13P3,-4,C10H12N5O13P3,c
h_c,H+,1,H,c
f6p_c,D-Fructose 6-phosphate,-2,C6H11O9P,c
h2o_c,H2O H2O,0,H2O,c
pi_c,Phosphate,-2,HO4P,c
fdp_c,"D-Fructose 1,6-bisphosphate",-4,C6H10O12P2,c


In [24]:
csv_met_cobra = os.path.join(data_dir,"gly_gluco_cobra_met.csv")
met_df.to_csv(csv_met_cobra)

In [25]:
rxn_df=pd.DataFrame()
for rxn in trial.reactions:
    cobra_rxn= trial.reactions.get_by_id(rxn.id)
    r=reaction_to_dict(cobra_rxn)
#     print(r)
    df=pd.DataFrame.from_dict(r,orient='index')
    df=df.T
    rxn_df=rxn_df.append(df)

rxn_df=rxn_df.set_index('id')
rxn_df

,name,metabolites,lower_bound,upper_bound,gene_reaction_rule,objective_coefficient
id,,,,,,
HEX1,Hexokinase (D-glucose:ATP),"{'adp_c': 1.0, 'atp_c': -1.0, 'g6p_c': 1.0, 'g...",0.0,1000.0,,NaN
PGI,Glucose-6-phosphate isomerase,"{'f6p_c': 1.0, 'g6p_c': -1.0}",-1000.0,1000.0,,NaN
FBP,Fructose-bisphosphatase,"{'f6p_c': 1.0, 'fdp_c': -1.0, 'h2o_c': -1.0, '...",0.0,1000.0,,NaN
PFK,Phosphofructokinase,"{'adp_c': 1.0, 'atp_c': -1.0, 'f6p_c': -1.0, '...",0.0,1000.0,,NaN
FBA,Fructose-bisphosphate aldolase,"{'dhap_c': 1.0, 'fdp_c': -1.0, 'g3p_c': 1.0}",-1000.0,1000.0,,NaN
TPI,Triose-phosphate isomerase,"{'dhap_c': -1.0, 'g3p_c': 1.0}",-1000.0,1000.0,,NaN
GAPD,Glyceraldehyde-3-phosphate dehydrogenase,"{'_13dpg_c': 1.0, 'g3p_c': -1.0, 'h_c': 1.0, '...",-1000.0,1000.0,,NaN
PGK,Phosphoglycerate kinase,"{'_13dpg_c': 1.0, '_3pg_c': -1.0, 'adp_c': 1.0...",-1000.0,1000.0,,NaN
PGM,Phosphoglycerate mutase,"{'_2pg_c': -1.0, '_3pg_c': 1.0}",-1000.0,1000.0,,NaN


In [33]:
json_rxn_cobra = os.path.join(data_dir,"gly_gluco_cobra_rxn.json")
rxn_df.to_json(json_rxn_cobra
,orient="index"
)
json_rxn_cobra

'c:\\Users\\sicil\\LiverModel\\data\\gly_gluco_cobra_rxn.json'

## Making MASS model of glycolysis

In [27]:
liver= MassModel("Core_Model", array_type='DataFrame', dtype=np.int64)

#### Add the pathway metabolites and their information to a new MASS model from COBRA model

In [34]:
import json
## make excluded list for reactions not needed in cc
# make a string split for 13dpg etc

In [35]:
# making dataframe of metabolites

met_csv_df = pd.read_csv(csv_met_cobra,index_col="id")
mass_metabolite_info=met_csv_df.to_dict(orient='index')

# Loading reaction data as JSON file to maintain data types as dictionaries 

with open(json_rxn_cobra) as json_file:
    mass_rxn_data = json.load(json_file)

# making dataframe of reactions
# csv_rxn = os.path.join(data_dir,"glycolysis_gluconeogenesis_reaction_df.csv")
# rxn_csv_df = pd.read_csv(csv_rxn,index_col="id") 
# reaction_info = rxn_csv_df.to_dict(orient='index')

In [36]:
#  Add all the remaining metabolites involved in the pathway
for met_id, info in mass_metabolite_info.items():
    met = MassMetabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    liver.add_metabolites(met)

In [37]:
for reaction, info in mass_rxn_data.items():
    reaction_obj = MassReaction(reaction)
    reaction_obj.id = reaction
    reaction= reaction_obj.id
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
#     if reaction_obj.id != "DM_nadh" or reaction_obj.id != 'ATPM':
    liver.add_reactions(reaction_obj)
    temp=info['metabolites']
    reaction_obj.add_metabolites(temp)
    print(reaction)

HEX1
PGI
FBP
PFK
FBA
TPI
GAPD
PGK
PGM
ENO
PEPCKm
PYK
PCm
LDH_L
PYRt2m
ADK1
ATPM
DM_nadh
PEPtm
G6PP
EX_glc__D_c
EX_pyr_c
EX_h2o_c
EX_h_c
EX_pi_c
EX_lac__L_c


In [38]:
trial.reactions.HEX1.lower_bound

0.0

In [39]:
liver.reactions.PGK.reverse_stoichiometry(inplace=True)
liver.reactions.PGM.reverse_stoichiometry(inplace=True)
# liver.reactions.PGM

Reaction identifier,PGM
Name,Phosphoglycerate mutase
Memory address,0x024920fc5e50
Subsystem,
Kinetic Reversibility,True
Stoichiometry,_3pg_c <=> _2pg_c 3-Phospho-D-glycerate <=> D-Glycerate 2-phosphate
GPR,
Bounds,"(-1000.0, 1000.0)"


In [40]:
liver.reactions.PFK

Reaction identifier,PFK
Name,Phosphofructokinase
Memory address,0x024949cb9e50
Subsystem,
Kinetic Reversibility,True
Stoichiometry,"atp_c + f6p_c <=> adp_c + fdp_c + h_c ATP C10H12N5O13P3 + D-Fructose 6-phosphate <=> ADP C10H12N5O10P2 + D-Fructose 1,6-bisphosphate + H+"
GPR,
Bounds,"(0.0, 1000.0)"


### Getting Standard and Physiological Gibbs energies of reactions

In [41]:
reaction_str= []

cc = ComponentContribution()

# optional: changing the aqueous environment parameters
cc.p_h = Q_(7.4)
cc.p_mg = Q_(3.31)
cc.ionic_strength = Q_("0.144M")
cc.temperature = Q_("310.15K")
R = 0.00831446261815324 #R = 0.00831446261815324 kJ/mol
from numpy import exp as exp
Keq_df=pd.DataFrame(columns=["Reaction","Keq"])
Keq_df

,Reaction,Keq


In [42]:


for reaction in liver.reactions:
    rxn_name = reaction.id        
    if rxn_name != "ATPM":
        reaction_str = reaction.reaction.split(" <=> ")
        reactants = reaction_str[0]
        reactants = reactants.split(" + ")

        for i in range(len(reactants)):
            string = reactants[i]
            l = len(string)
            reactants[i] = string[:l-2]
            reactants[i]= "bigg.metabolite:" + reactants[i]
        reactants_string= " + ".join(reactants)
        reactants_string = str(reactants_string + " = ")   



        products = reaction_str[1]
        products = products.split(" + ")

        for i in range(len(products)):
            string = products[i]
            l = len(string)
            products[i] = string[:l-2]            
            products[i]= "bigg.metabolite:" + products[i]
        products_string= ' + '.join(products)
        reaction_to_parse = reactants_string + products_string         
        rxn_parsed = cc.parse_reaction_formula(reaction_to_parse)

        print(rxn_name)
        dG0_prime = cc.standard_dg_prime(rxn_parsed)
    #         print(f"ΔG'° = {dG0_prime}")

        dGm_prime = cc.physiological_dg_prime(rxn_parsed)
        dG_prime_value_in_kj_per_mol = dGm_prime.value.m_as("kJ/mol")
        dG0prime_value_in_kj_per_mol = dG0_prime.value.m_as("kJ/mol")
    #     print(f"ΔG'm = {dG_prime_value_in_kj_per_mol}")
        delG = dG_prime_value_in_kj_per_mol
        a = exp( -delG/ (R*310.15)) 
        reaction.Keq = a
        print(f"Keq = {a}")
#         list_keq=[reaction,a]
#         rxn_keq=pd.DataFrame(list_keq)
        df = pd.DataFrame([[reaction,a]], columns=["Reaction","Keq"])
        Keq_df.append(df)
    else:
        reaction.Keq=float("inf")
# Keq_df    

HEX1
Keq = 4959.419292148073
PGI
Keq = 0.34798040939490554
FBP
Keq = 115004.76795942851
PFK
Keq = 1454.9063091036724
FBA
Keq = 0.1620181301040788
TPI
Keq = 0.11365701527311964


ParseException: bigg.metabolite:_13dpg was not found in the compound cache

In [ ]:
# Keq

In [ ]:
#adding DM_NADH 
nadhm = {'nadh_c': -1, 'h_c': 1, 'nad_c': 1}
rxn = mass.MassReaction('DM_nadh')
liver.add_reaction(rxn)
rxn.add_metabolites(nadhm)


In [ ]:
liver.reactions.DM_nadh.Keq=float("inf")

In [ ]:
# atpm = {'atp_c': -1,'h2o_c': -1,
#     'adp_c': 1,'h_c': 1,'pi_c': 1}
# rxn_2 = mass.MassReaction("ATPM")
# liver.add_reaction(rxn_2)
# rxn.add_metabolites(atpm)


In [ ]:
# liver.add_reactions([ATPM, DM_nadh])
# ATPM.Keq

In [ ]:
for met in [
#     'co2_m', 'gdp_m', 'gtp_m', 'pi_m', 'hco3_m','h_m','atp_c','atp_m'
    'glc__D_c','h_c','h2o_c','pi_c','pyr_c','lac__L_c'
#     ,'nadh_c', 'nad_c'


           ]:
    rxn = MassReaction('EX_%s' % met)
    liver.add_reaction(rxn)
    rxn.add_metabolites({met: -1})
    if met == 'glc__D_c':
        rxn.lower_bound = -1 # negative exchange flux = uptake
    elif met == 'pyr_c':
        rxn.lower_bound = 0
    else:
        rxn.lower_bound = -1000

##  Set Fluxes

In [ ]:
reaction_list=[]
for reaction in trial.reactions:
    r_id = reaction.id
    reaction_list.append(r_id)
# reaction_list.append('DM_nadh')
reaction_list

In [ ]:
# making dataframe of reactions

csv_minspan_dir = os.path.join(minspan_dir,"liver_csv_minspan.csv")
minspan_df = pd.read_csv(csv_minspan_dir,index_col="rxn") 

In [ ]:
# minspan_df.

In [ ]:
# solved_fluxes = minspan(liver, cores=3, verbose=False, timelimit=60)

# df = pd.DataFrame(solved_fluxes.copy(), index=[i.id for i in model.reactions])
# df = df/df.abs().max()
# csv = os.path.join(data_dir,"minspan_trial 1")
# df.to_csv(csv  % model_name)

In [ ]:
for reaction, flux in flux_solution[reaction_list].iteritems():
    reaction = liver.reactions.get_by_id(reaction)
    reaction.steady_state_flux = flux 
#     * 1278.605 / 1000 # mmol/gdW*hr --> mmol/L * hr ---> mol/L*hr
    print("{0}: {1}".format(reaction.flux_symbol_str,
                            reaction.steady_state_flux))

In [ ]:
liver.metabolites.h_c.charge= 1
# liver.metabolites.h_e.charge= 1
liver.metabolites.pi_c.charge= -2

In [ ]:
#Function to add underscore in front of metabolite identifiers which start with a number
## fix this only after getting Keq from equlibrator because bigg doesn't have _13dpg for example
def prefix_number_id(id_str):
    """Prefix identifiers that start with numbers."""
    if re.match(r"^\d", id_str):
        id_str = "_" + id_str
    return id_str

In [ ]:
#Loop to edit the names using "prefix_number_id" function defined earlier
for metabolite in liver.metabolites:
    new_met_id = prefix_number_id(metabolite.id)
    metabolite.id = new_met_id
liver.repair()

## Setting initial conditions

In [ ]:
import csv
csv_ic = os.path.join(data_dir,"Concentrations_Glycolysis.csv")
# opening the CSV file
with open(csv_ic, mode ='r')as file:
   
  # reading the CSV file
    csvFile = csv.reader(file)

      # displaying the contents of the CSV file
    for name, value in csvFile:
        met_id = liver.metabolites.get_by_id(name)
        met_id.ic = float(value) #M
        print(met_id, ":", met_id.ic)
        print(name,value)

## Set H2O/H Constants

In [ ]:
# Set concentrations of hydrogen, water as fixed
# Assume hydrogen in all compartments are fixed 
for metabolite in [
    "h_c",
#     'h_e','h_m',
#     'h2o_r','h2o_e',
    "h2o_c",
#     "glc__D_e", "pi_e",'pi_m',
    'pi_c'
                  ]:
    metabolite = liver.metabolites.get_by_id(metabolite)
    metabolite.fixed = True

In [ ]:
for metabolite in [
#     'h_m',
"h_c", 
#     'h_e',
"h2o_c",
#     'h2o_r','h2o_e' 
]:
    metabolite = liver.metabolites.get_by_id(metabolite)
    metabolite.ic = 1 #M

In [ ]:
for metabolite in liver.metabolites:
    print("{0}: {1}".format(metabolite.id, metabolite.ic))

In [ ]:
#Formulate QP minimization list for concentrations
conc_solver = ConcSolver(
    liver, 
    excluded_metabolites=[
    "h_c",
#         'h_e', 'h_m',
#     'h2o_r','h2o_e',
        "h2o_c",
#     "pi_e",
#     "glc__D_e", "lac__L_e",
#     "pyr_e"
    ], 
#     excluded_reactions=["Ht", "H2Ot", "H2Oter",'PIter','PIt',
#     'G6Pter','GLCter','GLCt1','PYRt2m','PEPtm',
#     'L_LACt2r','PYRt2'],

    constraint_buffer=1)

conc_solver.setup_feasible_qp_problem(
fixed_conc_bounds=list(liver.fixed))
# fixed_Keq_bounds=liver.reactions.list_attr("Keq_str")
    
#assumption: provide uncertainity to allow for a variance for the Keqs rather than staying fixed
#conc_solver.setup_feasible_qp_problem(Keq_percent_deviation=0.1)

conc_solution = conc_solver.optimize()
conc_solution

In [ ]:
#Function to load data from the excel sheet
def load_data(filepath, sheet_name):
    """Load Liver data from an excel sheet"""
    df = pd.read_excel(engine='openpyxl',
        io=filepath,
        sheet_name=sheet_name,
        index_col=0)
    return df

In [ ]:
#Compare Results
csv_ic = os.path.join(data_dir,"Concentrations_Glycolysis.xlsx")
# ic_df = pd.read_csv(csv_ic)
# ic_df.set_index()
ic_info = load_data(
    filepath=csv_ic,
    sheet_name="Concentrations")

In [ ]:
conc_solution.Keqs

In [ ]:
liver.parameters

In [ ]:
conc_comparison_fig, ax = plt.subplots(nrows=1, ncols=1,
                                       figsize=(5, 5))

plot_comparison(
    x=ic_info["Concentrations in M"], y=conc_solution,
    compare="concentrations",
    observable=[mid for mid in ic_info.index], ax=ax,
    legend="right outside", plot_function="loglog",
    xlim=(1e-6,0.02), ylim=(1e-6,1e-1),
    xy_line=True,
    xy_legend="best", xlabel="Initial [mol/L]", ylabel="Adjusted [mol/L]")

conc_comparison_fig.tight_layout()
update_model_with_concentration_solution(
    liver, conc_solution, concentrations=True, inplace=True);

In [ ]:
# Keq_data = pd.read_csv("./data/analysis_data/equilibrium_constants.csv", index_col=0)

# Keq_data_comparison_fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
# plot_comparison(
#     x=Keq_data["Initial"], y=Keq_data["Adjusted"], ax=ax,
#     title="Equilibrium Constants comparison",
#     legend="right outside", plot_function="loglog",
#     xlim=(1e-5, 1e5), ylim=(1e-5, 1e5), 
#     xy_line=True,
#     xy_legend="best", xlabel="Initial", ylabel="Adjusted")

# Keq_data_comparison_fig.tight_layout()

In [ ]:
liver.reactions.EX_glc__D_c.Keq = 1
liver.reactions.EX_h_c.Keq = 1 
liver.reactions.EX_h2o_c.Keq = 1 
liver.reactions.EX_pi_c.Keq = 1 
liver.reactions.EX_pyr_c.Keq = 1 
liver.reactions.EX_lac__L_c.Keq = 1 
# liver.reactions.EX_nadh_c.Keq = 1 
# liver.reactions.EX_nad_c.Keq = 1 

In [ ]:
liver.boundary_conditions['glc__D_b']=0.010482807 ##M ## initial condition of the metabolite in the model
liver.boundary_conditions['h2o_b']=1 ##M
liver.boundary_conditions['h_b']=0.0064 ##M
liver.boundary_conditions['pi_b']=0.010482807 ##M
liver.boundary_conditions['pyr_b']=0.00048 ##M
liver.boundary_conditions['nadh_b']=4.63542E-06
liver.boundary_conditions['nad_b']=0.00182531
liver.boundary_conditions['lac__L_b']=0.003261838

In [ ]:
Boundary= [
             "EX_glc__D_c", 
            "EX_h_c", 
            "EX_h2o_c",
            'EX_pyr_c',
            'EX_pi_c',
    "ATPM",
    
#     'EX_nad_c',
    "EX_lac__L_c"
]
for x in Boundary:
    a=liver.reactions.get_by_id(x)
    a.kf=1e6

In [ ]:
liver.calculate_PERCs(fluxes={
                r: v for r, v in liver.steady_state_fluxes.items()
                if not (r.boundary  
#                         or r== liver.reactions.GLCter or r== glycogenolysis.reactions.PIter
#                                or r== glycogenolysis.reactions.G6Pter or r== glycogenolysis.reactions.PIt or 
#                                 r== glycogenolysis.reactions.PYRt2m or 
#                                 r== glycogenolysis.reactions.L_LACt2r or 
#                                 r== glycogenolysis.reactions.PYRt2 or 
#                                 r== glycogenolysis.reactions.PEPtm or r== glycogenolysis.reactions.GLCt1 
#                                 or r== glycogenolysis.reactions.Ht
                       )
},
                      update_reactions=True)

print("Forward Rate Constants\n----------------------")
for reaction in liver.reactions:
    print("{0}: {1:.6f}".format(reaction.kf_str, reaction.kf))

In [ ]:
qcqa_model(liver, parameters=True, concentrations=True,
           fluxes=True, superfluous=True, elemental=True)

In [ ]:
# Setup simulation object
simG=Simulation(liver, verbose=True)
# Simulate from 0 to 1000 with 10001 points in the output
conc_sol, flux_sol = simG.simulate(liver, time=(0, 1e8))
# Quickly render and display time profiles
conc_sol.view_time_profile()

In [ ]:
#function to set x label and y label
def set_x_label(xlabel):
    if xlabel==None:
        xlabel="Time"
    else:
        xlabel=xlabel
    return xlabel
        
def set_y_label(ylabel):        
    if ylabel==None:
        ylabel="Concentrations"
    else:
        ylabel=ylabel
    return ylabel


In [ ]:
#function to make plot typelinear log
def set_plot_type(plot_type):
    if plot_type == None:
        x_plot_type = 'log'
        y_plot_type = 'log'
    elif plot_type == "logx":
        x_plot_type = 'log'
        y_plot_type = 'linear'
    elif plot_type == "logy":
        x_plot_type = 'linear'
        y_plot_type = 'log'
    elif plot_type == "linear":
        x_plot_type = 'linear'
        y_plot_type = 'linear'
    elif plot_type == "logxlogy":
        x_plot_type = 'log'
        y_plot_type = 'log'
    return x_plot_type,y_plot_type


In [ ]:
def view_all_conc_sol(conc_sol,
#                       dynamic=True
                      xlim=None,ylim=None,plot_type=None,xlabel=None,ylabel=None,width=None,height=None):
    """Generate an interactable time profile which can zoom into the plot with interactive tooltip"""
    #Shift click to click multiple
    
    data=conc_sol.to_frame()
    data = data.reset_index().melt('Time', var_name='Metabolites', value_name='Concentrations')
    
    # Setting X limit and Y limit
    def set_ylim(ylim):
        if ylim == None:
            conc=data['Concentrations']
            ylim_max=max(conc)
            ylim=[1e-5,ylim_max]
        else:
            ylim=ylim
        return ylim
    
    def set_xlim(xlim):
        if xlim == None:
            xlim_max=data['Time'].iloc[-1]
            xlim=[1e-5,xlim_max]

        else:
            xlim=xlim
        return xlim
    xlim=set_xlim(xlim)
    ylim=set_ylim(ylim)
        
       
    # Setting type of plot
    x_plot_type=set_plot_type(plot_type)[0]
    y_plot_type=set_plot_type(plot_type)[1]
        
    # Setting x label and y label
    xlabel=set_x_label(xlabel)
    ylabel=set_y_label(ylabel)
    # Setting width and height
    if width==None:
        width=500
    else:
        width=width
        
    if height==None:
        height=300
    else:
        height=height
    
    alt.data_transformers.disable_max_rows()
    selection = alt.selection_multi(fields=['Metabolites'])
    color = alt.condition(selection,
                          alt.Color('Metabolites:N', legend=None),
                          alt.value('lightgray'))

    line = alt.Chart(data).mark_line(clip=True).encode(alt.X('Time:Q',
                                                             title=xlabel,
                                                             scale=alt.Scale(type = x_plot_type,
                                                                             padding = 10,
                                                                             domain = xlim,
#                   zero=True,
                              ),
              axis=alt.Axis(tickCount=5)),
        alt.Y('Concentrations:Q',
              title=ylabel,
              scale=alt.Scale(type=y_plot_type,
                              padding = 10,
                              domain = ylim,
#                   zero=True, 
                             ),
               axis=alt.Axis(grid=False,
                             tickCount=5)),
        color=color,
        tooltip= ["Metabolites"]+["Time"]+["Concentrations"]
    ).properties(width=width, height=height).interactive()

    
    
#         if dynamic == False:
#             A = alt.layer(line)
#         else:
    legend = alt.Chart(data).mark_point().encode(
        y=alt.Y('Metabolites:N', axis=alt.Axis(orient='right')),
        color=color
    ).add_selection(
        selection
    )
    A= line | legend
    return A

In [ ]:
view_all_conc_sol(conc_sol,plot_type='logxlogy')